In [4]:
f = open("demofile.txt", "r")
s = f.read()
res = ''
r = open("result.txt", "a")
table = []

state = 0
line = 1
word = 0
chars = 0
token = ''

def reverse(s: str) -> int:
    if s == 'int' or s == 'float' or s == 'const':
        return 1
    elif s == 'void' or s == 'if' or s == 'else' or s == 'while' or s == 'break' or s == 'continue' or s == 'return':
        return 2
    else: return 0

def table_insert(s: str, prev: int):
    for i in range(len(table)):
        if s == table[i]: return i + 1

    table.append(s)
    # if prev == 1: return len(table)
    # else: return -1
    return len(table)

for i in s:
    if i != '\n': chars += 1

i = 0
while i < len(s):
    if state == 0:
        word += 1
        while s[i] == " ": i += 1
        token = ''
        token += s[i]
        if (s[i] >= 'a' and s[i] <='z') or (s[i] >= 'A' and s[i] <= 'Z'):
            state = 1
        elif s[i] >= '0' and s[i] <= '9':
            state = 2
        elif s[i] == '<':
            state = 8
        elif s[i] == '>':
            state = 9
        elif s[i] == '=':
            state = 10
        elif s[i] == '!':
            state = 11
        elif s[i] == '/':
            state = 12
        elif s[i] == '+':
            res += "('+',-);"
        elif s[i] == '-':
            res += "('-',-);"
        elif s[i] == '*':
            res += "('*',-);"
        elif s[i] == '(':
            res += "('(',-);"
        elif s[i] == ')':
            res += "(')',-);"
        elif s[i] == '%':
            res += "('%',-);"
        elif s[i] == '[':
            res += "('[',-);"
        elif s[i] == ']':
            res += "(']',-);"
        elif s[i] == '{':
            res += "('{',-);"
        elif s[i] == '}':
            res += "('}',-);"
        elif s[i] == ';':
            res += "(';',-);"
        elif s[i]=='\n':
            line += 1
            word -= 1
            res += '\n'
        elif s[i] == '&':
            state = 15
        elif s[i] == '|':
            state = 16
        else: 
            state = 17
    elif state == 1:
        token += s[i]
        if (s[i] >= 'a' and s[i] <='z')\
              or (s[i] >= 'A' and s[i] <= 'Z')\
              or (s[i] >= '0' and s[i] <= '9')\
              or s[i] == '_':
            state = 1
        else:
            token = token[:-1]
            i -= 1
            state = 0
            iskey = reverse(token)
            if iskey:
                res = res + "('KEY'," + token + ");"
            else:
                identry = table_insert(token, iskey)
                iskey = 0
                # if identry != -1:
                #     res = res + "('ID'," + str(identry) + ");"
                # else:
                #     res = res + "at line:" + str(line) + "('ERROR',unknown var);"
                res = res + "('ID'," + str(identry) + ": " + token + ");"
    elif state == 2:
        token += s[i] 
        if s[i].isdigit():
            state = 2
        elif s[i] == '.':
            state = 3
        elif s[i] == 'e' or s[i] == 'E':
            state = 5
        else:
            token = token[:-1]
            i -= 1
            state = 0
            res = res + "('NUM'," + token + ");"
    elif state == 3:
        token += s[i]
        if s[i].isdigit():
            state = 4
        else:
            res = res + "at line:" + str(line) + "('ERROR',float);"
            state = 0
    elif state == 4:
        token += s[i]
        if s[i].isdigit():
            state = 4
        elif s[i] == 'e' or s[i] == 'E':
            state = 5
        else:
            token = token[:-1]
            i -= 1
            state = 0
            res = res + "('FLOAT'," + token + ");"
    elif state == 5:
        token += s[i]
        if s[i].isdigit():
            state = 7
        elif s[i] == '+' or s[i] == '-':
            state = 6
        else:
            i -= 1
            state = 0
            res = res + "at line:" + str(line) + "('ERROR',exp);"
    elif state == 6:
        token += s[i]
        if s[i].isdigit():
            state = 7
        else:
            i -= 1
            state = 0
            res = res + "at line:" + str(line) + "('ERROR',exp);"
    elif state == 7:
        token += s[i]
        if s[i].isdigit():
            state = 7
        else:
            token = token[:-1]
            i -= 1
            state = 0
            res = res + "('EXP'," + str(token) + ");"
    elif state == 8:
        token += s[i]
        if s[i] == '=':
            state = 0
            res += "('RELOP',le);"
        else:
            i -= 1
            state = 0
            res += "('RELOP',lt);"
    elif state == 9:
        token += s[i]
        if s[i] == '=':
            state = 0
            res += "('RELOP',ge);"
        else:
            i -= 1
            state = 0
            res += "('RELOP',gt);"
    elif state == 10:
        token += s[i]
        if s[i] == '=':
            state = 0
            res += "('RELOP',eq);"
        else:
            i -= 1
            state = 0
            res += "('ASSIGN',-);"
    elif state == 11:
        token += s[i]
        if s[i] == '=':
            state = 0
            res += "('RELOP',ne);"
        else:
            i -= 1
            state = 0
            res += "('LOGOP',not);"
    elif state == 12:
        token += s[i]
        if s[i] == '*':
            state = 13
        elif s[i] == '/':
            state = 14
        else:
            i -= 1
            state = 0
            res += "('/',-);"
    elif state == 13:
        word -= 1
        while i < len(s) and s[i] != '*':
            i += 1
        i += 1
        if s[i] == '/':
            state = 0
            res += "('COMMENT',-);"
        else: state = 13
    elif state == 14:
        word -= 1
        token += s[i]
        while i < len(s) and s[i] != '\n':
            i += 1
        i -= 1
        state = 0
        res += "('COMMENT',-);"
    elif state == 15:
        token += s[i]
        state = 0
        if s[i] == '&':
            res += "('LOGOP',and);"
        else:
            res = res + "at line:" + str(line) + "('ERROR',logop_and);"
    elif state == 16:
        token += s[i]
        state = 0
        if s[i] == '|':
            res += "('LOGOP',or);"
        else:
            res = res + "at line:" + str(line) + "('ERROR',logop_or);"     
    elif state == 17:
        state = 0
        res = res + "at line:" + str(line) + "('ERROR',unknown);"
    i += 1

res = res + "\n" + "Total lines:" + str(line) + "\n"
res = res + "Total words(except comment):" + str(word) + "\n"
res = res + "Total chars:" + str(chars) + "\n"

# res += '----test table----\n'
# for i in table:
#     res = res + str(i) + "\n"

r.write(res)

1058

In [2]:
r = open("result.txt", "w")
r.write("")

0

In [ ]:
f = open("demofile.txt", "r")
s = f.read()
print(s)